#  Sytematic Review for "Gateway to the perspectives of the Food-Energy-Water Nexus"

If you have any questions please contact Kyle Proctor (proctork@oregonstate.edu) or Ganti S. Murthy (ganti.murthy@oregonstate.edu)

This code was used to conduct a systematic review to identify publishing trends related to the FEW nexus. The review looks at how the quantity of FEW nexus studies has changed over the years, where those studies were coming from, and which journals they were being published in. 

The analysis was conducted using the Pyscopus plugin for python (http://zhiyzuo.github.io/python-scopus/). Pyscopus is a wrapper for the scopus API; the world’s largest database of peer reviewed literature. 

## Use of this code requires a scopus API KEY 
scopus API keys can be obtained at this link (https://dev.elsevier.com/) a variable named "key" must then be created which is the value of the scopus key enclosed in single quotation marks
for example : key = 'yourscoupuskeyhere' ( be sure there are no spaces within the quotation marks)

The analysis provided here will produce all papers which match the corresponding keywords and were produced in 2011 or later. The analysis used in the journal article was conducted at the end of July and thus shows all papers produced before August 1st 2019
The majorority of the work here is done by the pyscopus software which returns a dataframe with all of the papers, the remainder of the code is data manipulation for visualization/

In [ ]:
#Install and import pyscopus plug in and other usefull libraries
import sys
!{sys.executable} -m pip install pyscopus
!{sys.executable} -m pip install requests
!{sys.executable} -m pip install openpyxl
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install matplotlib
import pyscopus
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 

In [ ]:
#input scopus API key (get one here https://dev.elsevier.com/)
from pyscopus import Scopus
key='' # you need to enter your own scopus API key here
scopus= Scopus(key);


In [ ]:
#This function is where all the analysis is conducted, It produces 2 charts and 3 tables. 
# Chart 1 The papers published by year and corresponding table
# Chart 2 The papers publihed by country and corresponding table
# Table 3 The papers published by Journal name
# The function can be used on any dataset produced by pyscopus


def Systematic_Review(Search_Dataset): # Search_Dataset set will be the results from pyscopus
    
    # these lines convert the cover date of the article into simply the year of the article and convert from str type
    # to intereger for later analysis     
    cover_date=Search_Dataset['cover_date']
    pubYear=cover_date.str.split('-',n=1, expand=True)
    Search_Dataset['cover_date']=pubYear
    year=Search_Dataset['cover_date'].astype(int)
    publisherData=Search_Dataset['publication_name'] # aquire Name of Journal article was published in
   
    # The affiliation column has a large amount of information about 
    # where the paper came from, university, city, etc. in this case we only want information about the 
    # Country of the first Authors institution, these lines get that data and save it as CD
    location=Search_Dataset['affiliation']; location = location.astype(str);location.dropna(inplace = True) 
    country=location.str.split("country",n=1,expand=True);country1=country[1];countryT=country1.str.split("}",n=1,expand=True)
    CountryData=countryT[0]
    CD=CountryData.str.slice(4, -1)
    CD=CD[CD!='on'] #removing any areas which have "none" listed as the country  
    
    # create a table showing how many papers were published each year
    pub_count=year.value_counts() #Count of how many publications were published each year
    pub_Perc=pub_count/len(year) # percentage of publications published in this year
    Year_published = np.transpose(np.array(pub_Perc.index)) 
    pub_table=pd.DataFrame(np.c_[Year_published,pub_count,pub_Perc],columns=["Year","count","percent"]) 
    
    # plot histogram of publications by year
    plt.title('Publications by Year')
    plt.xlabel('Year')
    plt.ylabel('# of publications')
    
    # Ensure that each year has its own Bin, starting from 2011 until the current year - note if the search
    # timeframe is changed these lines must also be
    bins=np.linspace(2011,(max(year)+1),(max(year)-2011+2)) 
    plt.xticks(bins)
    hist=plt.hist(year,bins=bins ,rwidth=0.5)
    plt.xlim([2010,(max(year)+1)])
    plt.show()
    
    display(pub_table)
    
    
    
    #plot the number of publications coming from different countries
    plt.figure(figsize=(11,11))
    plt.title('Publications by Country')
    plt.xlabel('Country of First Author institution')
    plt.ylabel('# of publications')
    Country_plot=pd.Series(CD).value_counts().plot('bar')
    plt.show()
    
    #Create a table with the papers published from each country
    #this table is not displayed because it is very large, to display simply uncomment
    CD_count=CD.value_counts()
    CD_index= np.transpose(np.array(CD_count.index))
    CD_Perc=CD_count/len(CD)
    CD_table=pd.DataFrame(np.c_[CD_index,CD_count,CD_Perc],columns=["Country","count","percent"])
    #display(CD_table)
    
    # Create a table with the papers published in each Journal
    
    JCount=publisherData.value_counts()
    JPerc=publisherData.value_counts()/len(CD)
    JIndex=np.transpose(np.array(JCount.index))
    JournalTable=pd.DataFrame(np.c_[JIndex,JCount,JPerc], columns=['Journal Name',"count","percent"])
    display(JournalTable)

    

## All Nexus Papers

In [ ]:

# This is where the search for papers occures, details on the Keywords used here are in the text
#D etails about the search syntax detailed here (https://dev.elsevier.com/tips/ScopusSearchTips.htm)

mainDataSet = scopus.search("TITLE-ABS-KEY(Nexus w/10 Energy AND Water w/10 food) OR TITLE-ABS-KEY(Nexus w/10 food AND energy w/10 food)   PUBYEAR > 2010", count=1000, view='STANDARD')

In [ ]:
pd.set_option('display.max_colwidth', -1) # ensure full values can be seen
display(mainDataSet) 
mainDataSet.to_csv("data.csv")

In [ ]:
Systematic_Review(mainDataSet)

## Ecosystem Perspective 

In [ ]:
EcosystemDS= scopus.search("TITLE-ABS-KEY(Nexus w/10 Energy AND Water w/10 food) OR TITLE-ABS-KEY(Nexus w/10 food AND energy w/10 food) AND (KEY(LCA) OR KEY(ecosystem) OR KEY(ecological indicators))  PUBYEAR > 2010", count=1000, view='STANDARD')
display(EcosystemDS)

In [ ]:
Systematic_Review(EcosystemDS)

## Waste Management Perspective

In [ ]:
WasteDS= scopus.search("TITLE-ABS-KEY(Nexus w/10 food AND Energy w/10 Water) AND (KEY(Waste) OR KEY(Circular Economy)) PUBYEAR > 2010", count=1000, view='STANDARD')
display(WasteDS)

In [ ]:
Systematic_Review(WasteDS)

## Learing Process Perspective

In [ ]:
LearningDS= scopus.search("TITLE-ABS-KEY(Nexus w/10 food AND Energy w/10 Water) AND TITLE-ABS-KEY(Learning Process OR Semiotic)  PUBYEAR > 2010", count=1000, view='STANDARD')
display(LearningDS)

In [ ]:
Systematic_Review(LearningDS)

## Institutional Change Perspective

In [ ]:
InstitutionalDS= scopus.search("KEY(policy OR institutions OR governance OR government)  AND TITLE-ABS-KEY(Nexus w/10 food AND Energy w/10 Water)  PUBYEAR > 2010", count=1000, view='STANDARD')
display(InstitutionalDS)

In [ ]:
Systematic_Review(InstitutionalDS)

## Stakeholder Trust Perspective

In [ ]:
TrustDS= scopus.search("KEY( Stakeholder) OR KEY(critical social science) AND TITLE-ABS-KEY(Nexus w/10 food AND Energy w/10 Water)  PUBYEAR > 2010", count=1000, view='STANDARD')
display(TrustDS)

In [ ]:
Systematic_Review(TrustDS)